In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
to_run = True
if to_run:
    !git clone https://github.com/nackta/text_classification.git # /content/drive/[your path]/text_classification
    # %cd /content/drive/[your path]
    %cd text_classification
    to_run = False
    # !wandb login

Cloning into 'text_classification'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 49 (delta 16), reused 44 (delta 11), pack-reused 0
Unpacking objects: 100% (49/49), done.
/content/text_classification


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 13.0 MB/s 
     |████████████████████████████████| 6.6 MB 45.0 MB/s 
     |████████████████████████████████| 596 kB 11.1 MB/s 
     |████████████████████████████████| 101 kB 5.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import os
import pickle

import pandas as pd
import numpy as np

import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

from dataset import *
from trainer import Trainer

In [6]:
open_file = open(os.path.join("./data/train_pos.pkl"), "rb")
train_pos = pickle.load(open_file)
open_file.close()

open_file = open(os.path.join("./data/train_neg.pkl"), "rb")
train_neg = pickle.load(open_file)
open_file.close()

open_file = open(os.path.join("./data/dev_pos.pkl"), "rb")
dev_pos = pickle.load(open_file)
open_file.close()

open_file = open(os.path.join("./data/dev_neg.pkl"), "rb")
dev_neg = pickle.load(open_file)
open_file.close()

open_file = open(os.path.join("./data/test.pkl"), "rb")
test = pickle.load(open_file)
open_file.close()

#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

train_dataset = SentimentDataset(tokenizer, train_pos, train_neg)
dev_dataset = SentimentDataset(tokenizer, dev_pos, dev_neg)
test_dataset = SentimentTestDataset(tokenizer, test)

train_batch_size=128
eval_batch_size=64
test_batch_size = 32

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size,
                                           shuffle=True, collate_fn=collate_fn_style,
                                           pin_memory=True, num_workers=2)

dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=eval_batch_size,
                                         shuffle=False, collate_fn=collate_fn_style,
                                         num_workers=2)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size,
                                          shuffle=False, collate_fn=collate_fn_style_test,
                                          num_workers=2)

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [7]:
random_seed=42
np.random.seed(random_seed)
torch.manual_seed(random_seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
model.to(device)
learning_rate = 5e-5

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [8]:
trainer = Trainer(train_loader=train_loader, dev_loader=dev_loader, test_loader=test_loader, model=model, lr=learning_rate, optimizer=AdamW, device=device)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [9]:
trainer.train(train_epoch=1)

Epoch 0:   2%|▏         | 59/3463 [26:19<25:19:07, 26.78s/batch, loss=0.498]


KeyboardInterrupt: ignored

In [ ]:
predictions=trainer.test()

In [ ]:
if not os.path.isdir("submission"):
    os.mkdir("submission")
test_df = pd.read_csv('./data/test_no_label.csv')
test_df['Category'] = predictions
test_df.to_csv('./submission/submission_test.csv', index=False)